# Phase 4: Data Mining

## Imports, Helper Functions

In [2]:
from cProfile import label
from itertools import count
from statistics import mean
from turtle import color
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
import psycopg2
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sqlalchemy import create_engine

country_codes = ['CAN', 'USA', 'MEX', 'IRN', 'THA', 'UKR', 'CAF', 'TCD', 'MLI']

countries_by_class_label = {
      'developed': ['CAN', 'USA'],
      'developing': ['MEX', 'IRN', 'THA', 'UKR'],
      'underdeveloped': ['CAF', 'TCD', 'MLI']
}

def get_country_development_class(country_code):
      if country_code in countries_by_class_label['developed']:
            return "developed"
      elif country_code in countries_by_class_label['developing']:
            return "developing"
      elif country_code in countries_by_class_label['underdeveloped']:
            return "underdeveloped"
      else:
            return np.NaN

def get_country_shape(country_code):
      if country_code in countries_by_class_label['developed']:
            return "s"
      elif country_code in countries_by_class_label['developing']:
            return "o"
      else:
            return "^"
      
def get_country_color(country_code):
      if country_code in countries_by_class_label['developed']:
            return "limegreen"
      elif country_code in countries_by_class_label['developing']:
            return "gold"
      else:
            return "red"

## Pulling Data from Dart Mart (PostgreSQL)

In [3]:
country_codes = ['CAN', 'USA', 'MEX', 'IRN', 'THA', 'UKR', 'CAF', 'TCD', 'MLI']

# SQL query to pull data from Postgres for data mining
query_pull_data = '''
SELECT DISTINCT 
       C.country_code, D.year_, 
       F.quality_of_life_index, F.human_development_index, F.income_index,
       C.total_population, C.population_growth, C.gdp_constant_2015, C.gdp_growth,
       ED.literacy_rate, ED.government_expenditure_on_education,
       H.birth_rate, H.death_rate, H.capital_health_expenditure, H.prevalence_of_overweight_female, H.prevalence_of_overweight_male, H.prevalence_of_hypertension_female, H.prevalence_of_hypertension_male,
       P.life_expectancy_female, P.life_expectancy_male, P.life_expectancy, P.net_migration, P.labor_force_total, P.unemployment_rate,
       LV.access_electricity, LV.coverage_social_insurance_programs, LV.access_to_fuels_for_cooking
FROM "CSI4142".fact_table as F, "CSI4142".country as C, "CSI4142".date_ as D, "CSI4142".education as ED, "CSI4142".health as H, "CSI4142".living_conditions as LV, "CSI4142".population_ as P
WHERE F.country_surrogate=C.surrogate_key AND
      F.date_surrogate=D.surrogate_key AND
      F.education_surrogate=ED.surrogate_key AND
      F.health_surrogate=H.surrogate_key AND
      F.living_conditions_surrogate=LV.surrogate_key AND
      F.population_surrogate=P.surrogate_key
ORDER BY C.country_code, D.year_
;
'''

# Pull data into dataframe
db_engine = create_engine('postgresql+psycopg2://bazzadev@localhost/bazzadev')
df = pd.read_sql_query(query_pull_data, db_engine)